In [1]:
import numpy as np
import pandas as pd
import yfinance as yfin
import matplotlib.pyplot as plt
from sdocks.consts import InvestmentData
from sdocks.performance import PerfMetrics
from plotly import graph_objects as go
from pandas_datareader import data as web

In [2]:
yfin.pdr_override()
benchmark_data=web.DataReader(
    'SPY',start='2015-12-31',end='2020-01-01'
).reset_index()

[*********************100%%**********************]  1 of 1 completed


In [3]:
# data=pd.read_csv('AWU.csv')
data=web.DataReader(
    'META',start='2015-12-31',end='2020-01-01'
).reset_index()

eod_data=data[['Date','Close']]
eod_data.columns=['date','close']
dates=[str(v)[:10] for v in list(eod_data.date.values)]
prices=list(eod_data.close.values)

[*********************100%%**********************]  1 of 1 completed


In [4]:
investment_data=InvestmentData(dates, prices)
investment_data.get_return()
investment_data.get_log_return()
data=investment_data.get_data()

In [5]:
benchmark_data=InvestmentData(
    [str(v)[:10] for v in benchmark_data['Date'].values], 
    list(benchmark_data['Close'].values)
)
benchmark_data.get_log_return()
benchmark_data=benchmark_data.get_data()

In [6]:
data=investment_data.get_data()
data=data[data.date >= '2016-01-01']
perf=PerfMetrics(
    data,
    0,
    benchmark_log_return=benchmark_data.dropna()['log return'].values,
    drawdown_method='percent',
    calmer_years=10
)
payload=perf.get_payload()

In [7]:
payload

{'volatility': 0.28814144535810976,
 'cagr': 0.19095296539024864,
 'sharpe': 0.6627056553871564,
 'downside_deviation': 0.20347510726989393,
 'sortino_ratio': 0.9384586053416567,
 'max_drawdown': {'max_drawdown': 0.42960917949676514,
  'peak_date': numpy.datetime64('2018-07-25T00:00:00.000000000'),
  'peak_price': 217.5,
  'trough_date': numpy.datetime64('2018-12-24T00:00:00.000000000'),
  'trough_price': 124.06},
 'log_max_drawdown_ratio': 0.13566804,
 'calmer_ratio': 0.16805355123600996,
 'pps': 0.12890557308698322,
 'jensens_alpha': 0.00011565018327158098}

In [ ]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(data.date.values),
        y=data.price.values,
        name='prices'
    )
)
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(
            [payload['max_drawdown']['trough_date'], payload['max_drawdown']['peak_date']]
        ),
        y=[
            payload['max_drawdown']['trough_price'], payload['max_drawdown']['peak_price']
        ],
        name=f"Max Drawdown: {payload['max_drawdown']['max_drawdown']*100:.2f} %"
    )
)
fig.update_layout(
    dict(
        title='AWU'
    )
)